In [17]:
import datetime

import time
import os
import sys
from pathlib import Path

import numpy as np
from scipy.stats import lognorm
import pandas as pd

from astropy import stats
from astropy.io import fits
from astropy.coordinates import EarthLocation
from astropy.time import Time
from astropy.utils import iers
import astropy.units as u
import astroplan

import matplotlib
#matplotlib.use('nbagg')
from matplotlib import style
style.use('ggplot')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [18]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
%cd /home/halcoll/halcoll/halcoll/data

/home/halcoll/halcoll/halcoll/data


In [20]:
from astropy.utils import iers
iers.IERS_A_URL = 'ftp://cddis.gsfc.nasa.gov/pub/products/iers/finals2000A.all'
astroplan.download_IERS_A()

In [21]:
temp_data = pd.read_csv("halcoll_temps.csv")
temp_data = temp_data.rename(columns={"temptrax1_probe7": "temptrax_outside", "temptrax1_probe8": "temptrax_chamber"})
temp_data.head()

,ts,cell_e_series_backplate_C,cell_e_series_chamber_ambient_C,cell_e_series_frontplate_C,cell_e_series_in_front_of_primary_C,cell_e_series_lower_plenum_C,cell_e_series_midplate_C,cell_e_series_outside_ambient_C,yankee_temperature,temptrax1_probe2,temptrax1_probe3,temptrax1_probe4,temptrax1_probe6,temptrax_outside,temptrax_chamber,temptrax3_probe10,temptrax3_probe11,temptrax3_probe12
0,2016-01-01 00:00:00,1.82905,1.71735,1.7748,1.6990,1.78825,1.8217,1.54470,1.633,1.8,1.8,2.0,1.8,1.8,2.1,1.7,1.9,1.8
1,2016-01-01 00:05:00,1.82340,1.68600,1.7659,1.6806,1.76570,1.8120,1.53410,1.588,1.7,1.8,1.9,1.8,1.8,2.0,1.7,1.9,1.8
2,2016-01-01 00:10:00,1.82435,1.70410,1.7618,1.6727,1.75620,1.8087,1.52885,1.658,1.8,1.8,1.9,1.9,1.8,2.1,1.7,1.8,1.8
3,2016-01-01 00:15:00,1.81050,1.71710,1.7481,1.6543,1.75790,1.7965,1.54970,1.724,1.7,1.8,1.9,1.8,1.8,2.1,1.7,1.9,1.9
4,2016-01-01 00:20:00,1.81450,1.76420,1.7586,1.6753,1.78190,1.8041,1.61250,1.723,1.8,1.8,1.9,1.8,1.9,2.1,1.7,1.8,1.9


In [22]:
lat = "31:41:19.6"
lon = "-110:53:04.4"
elevation = 2600 * u.m
location = EarthLocation.from_geodetic(lon, lat, elevation)
mmt = astroplan.Observer(name="MMTO", location=location, timezone="US/Arizona", pressure=0*u.mbar)
tset = mmt.sun_set_time(Time(datetime.datetime.now()), which='next', horizon=-0.8333*u.deg)
tset.isot

'2019-11-22T00:21:42.333'

In [23]:
tset_r = tset + datetime.timedelta(seconds=30)
tset_r.strftime("%H %M")

'00 22'

In [24]:
from skyfield import api
ts = api.load.timescale()
e = api.load('de421.bsp')
from skyfield import almanac

In [25]:
mmt_sf = api.Topos(latitude_degrees=location.lat.value, longitude_degrees=location.lon.value, elevation_m=2600.0)

In [26]:
t0 = ts.utc(2016, 1, 1, 0)
t1 = ts.utc(2019, 12, 31, 23)
t, y = almanac.find_discrete(t0, t1, almanac.sunrise_sunset(e, mmt_sf))

In [27]:
t_str = np.array(t.utc_iso(), dtype=np.unicode_)
t_dt = np.array(t.utc_datetime())

In [28]:
# y is true when sun is up, false when down. so sunset is when y is False.
sunsets = t_str[y == False]
sunsets_dt = t_dt[y == False]
sunsets

array(['2016-01-01T00:30:04Z', '2016-01-02T00:30:47Z',
       '2016-01-03T00:31:31Z', ..., '2019-12-29T00:28:04Z',
       '2019-12-30T00:28:43Z', '2019-12-31T00:29:24Z'], dtype='<U20')

In [29]:
temp_data = temp_data.set_index(pd.DatetimeIndex(temp_data['ts'], name='mst')).tz_localize('MST').drop(columns=['ts'])

In [30]:
temp_data

,cell_e_series_backplate_C,cell_e_series_chamber_ambient_C,cell_e_series_frontplate_C,cell_e_series_in_front_of_primary_C,cell_e_series_lower_plenum_C,cell_e_series_midplate_C,cell_e_series_outside_ambient_C,yankee_temperature,temptrax1_probe2,temptrax1_probe3,temptrax1_probe4,temptrax1_probe6,temptrax_outside,temptrax_chamber,temptrax3_probe10,temptrax3_probe11,temptrax3_probe12
mst,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00-07:00,1.82905,1.71735,1.7748,1.6990,1.78825,1.8217,1.54470,1.6330,1.8,1.8,2.0,1.8,1.80,2.1,1.7,1.9,1.8
2016-01-01 00:05:00-07:00,1.82340,1.68600,1.7659,1.6806,1.76570,1.8120,1.53410,1.5880,1.7,1.8,1.9,1.8,1.80,2.0,1.7,1.9,1.8
2016-01-01 00:10:00-07:00,1.82435,1.70410,1.7618,1.6727,1.75620,1.8087,1.52885,1.6580,1.8,1.8,1.9,1.9,1.80,2.1,1.7,1.8,1.8
2016-01-01 00:15:00-07:00,1.81050,1.71710,1.7481,1.6543,1.75790,1.7965,1.54970,1.7240,1.7,1.8,1.9,1.8,1.80,2.1,1.7,1.9,1.9
2016-01-01 00:20:00-07:00,1.81450,1.76420,1.7586,1.6753,1.78190,1.8041,1.61250,1.7230,1.8,1.8,1.9,1.8,1.90,2.1,1.7,1.8,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-20 17:00:00-07:00,9.55270,10.63850,9.5911,9.8355,9.61180,9.4938,9.27890,8.3940,11.6,9.9,9.6,10.6,5.75,8.5,10.8,9.8,9.6
2019-11-20 17:05:00-07:00,9.55270,10.63850,9.5911,9.8355,9.61180,9.4938,9.27890,8.3120,11.6,9.9,9.6,10.6,5.80,8.5,10.8,9.8,9.6
2019-11-20 17:10:00-07:00,9.55270,10.63850,9.5911,9.8355,9.61180,9.4938,9.27890,8.3035,11.6,9.9,9.6,10.6,6.00,8.5,10.8,9.8,9.6


In [31]:
sunsets_df = pd.DataFrame(pd.to_datetime(sunsets).tz_convert("MST"), columns=['sunset'])
sunsets_df = sunsets_df.set_index(pd.DatetimeIndex(sunsets_df['sunset'], name='mst'))
sunsets_df

,sunset
mst,
2015-12-31 17:30:04-07:00,2015-12-31 17:30:04-07:00
2016-01-01 17:30:47-07:00,2016-01-01 17:30:47-07:00
2016-01-02 17:31:31-07:00,2016-01-02 17:31:31-07:00
2016-01-03 17:32:15-07:00,2016-01-03 17:32:15-07:00
2016-01-04 17:33:01-07:00,2016-01-04 17:33:01-07:00
...,...
2019-12-26 17:26:48-07:00,2019-12-26 17:26:48-07:00
2019-12-27 17:27:25-07:00,2019-12-27 17:27:25-07:00
2019-12-28 17:28:04-07:00,2019-12-28 17:28:04-07:00


In [32]:
sunset_temp_data = pd.merge_asof(temp_data, sunsets_df, on='mst', direction='forward')
sunset_temp_data = sunset_temp_data.set_index(pd.DatetimeIndex(sunset_temp_data['mst'], name='mst_i'))

In [33]:
sunset_temp_data['sunset_diff'] = sunset_temp_data['mst'] - sunset_temp_data['sunset']
sunset_temp_data['hrs_to_sunset'] = -1*(sunset_temp_data['sunset_diff'].dt.days * 86400 + sunset_temp_data['sunset_diff'].dt.seconds)/3600
sunset_temp_data

,mst,cell_e_series_backplate_C,cell_e_series_chamber_ambient_C,cell_e_series_frontplate_C,cell_e_series_in_front_of_primary_C,cell_e_series_lower_plenum_C,cell_e_series_midplate_C,cell_e_series_outside_ambient_C,yankee_temperature,temptrax1_probe2,...,temptrax1_probe4,temptrax1_probe6,temptrax_outside,temptrax_chamber,temptrax3_probe10,temptrax3_probe11,temptrax3_probe12,sunset,sunset_diff,hrs_to_sunset
mst_i,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00-07:00,2016-01-01 00:00:00-07:00,1.82905,1.71735,1.7748,1.6990,1.78825,1.8217,1.54470,1.6330,1.8,...,2.0,1.8,1.80,2.1,1.7,1.9,1.8,2016-01-01 17:30:47-07:00,-1 days +06:29:13,17.513056
2016-01-01 00:05:00-07:00,2016-01-01 00:05:00-07:00,1.82340,1.68600,1.7659,1.6806,1.76570,1.8120,1.53410,1.5880,1.7,...,1.9,1.8,1.80,2.0,1.7,1.9,1.8,2016-01-01 17:30:47-07:00,-1 days +06:34:13,17.429722
2016-01-01 00:10:00-07:00,2016-01-01 00:10:00-07:00,1.82435,1.70410,1.7618,1.6727,1.75620,1.8087,1.52885,1.6580,1.8,...,1.9,1.9,1.80,2.1,1.7,1.8,1.8,2016-01-01 17:30:47-07:00,-1 days +06:39:13,17.346389
2016-01-01 00:15:00-07:00,2016-01-01 00:15:00-07:00,1.81050,1.71710,1.7481,1.6543,1.75790,1.7965,1.54970,1.7240,1.7,...,1.9,1.8,1.80,2.1,1.7,1.9,1.9,2016-01-01 17:30:47-07:00,-1 days +06:44:13,17.263056
2016-01-01 00:20:00-07:00,2016-01-01 00:20:00-07:00,1.81450,1.76420,1.7586,1.6753,1.78190,1.8041,1.61250,1.7230,1.8,...,1.9,1.8,1.90,2.1,1.7,1.8,1.9,2016-01-01 17:30:47-07:00,-1 days +06:49:13,17.179722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-20 17:00:00-07:00,2019-11-20 17:00:00-07:00,9.55270,10.63850,9.5911,9.8355,9.61180,9.4938,9.27890,8.3940,11.6,...,9.6,10.6,5.75,8.5,10.8,9.8,9.6,2019-11-20 17:22:05-07:00,-1 days +23:37:55,0.368056
2019-11-20 17:05:00-07:00,2019-11-20 17:05:00-07:00,9.55270,10.63850,9.5911,9.8355,9.61180,9.4938,9.27890,8.3120,11.6,...,9.6,10.6,5.80,8.5,10.8,9.8,9.6,2019-11-20 17:22:05-07:00,-1 days +23:42:55,0.284722
2019-11-20 17:10:00-07:00,2019-11-20 17:10:00-07:00,9.55270,10.63850,9.5911,9.8355,9.61180,9.4938,9.27890,8.3035,11.6,...,9.6,10.6,6.00,8.5,10.8,9.8,9.6,2019-11-20 17:22:05-07:00,-1 days +23:47:55,0.201389


In [44]:
st_trim = sunset_temp_data[(sunset_temp_data['hrs_to_sunset'] > 0.) & (sunset_temp_data['hrs_to_sunset'] < 5.) & (sunset_temp_data['temptrax_outside'] > -10) & (sunset_temp_data['temptrax_outside'] < 35)]
#st_trim = st_trim.dropna()

In [45]:
st_trim

,mst,cell_e_series_backplate_C,cell_e_series_chamber_ambient_C,cell_e_series_frontplate_C,cell_e_series_in_front_of_primary_C,cell_e_series_lower_plenum_C,cell_e_series_midplate_C,cell_e_series_outside_ambient_C,yankee_temperature,temptrax1_probe2,...,temptrax1_probe4,temptrax1_probe6,temptrax_outside,temptrax_chamber,temptrax3_probe10,temptrax3_probe11,temptrax3_probe12,sunset,sunset_diff,hrs_to_sunset
mst_i,,,,,,,,,,,,,,,,,,,,,
2016-01-01 12:35:00-07:00,2016-01-01 12:35:00-07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.4,...,3.8,6.5,11.10,5.2,4.5,4.9,3.8,2016-01-01 17:30:47-07:00,-1 days +19:04:13,4.929722
2016-01-01 12:40:00-07:00,2016-01-01 12:40:00-07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.4,...,3.8,6.6,11.10,5.2,4.6,4.9,3.9,2016-01-01 17:30:47-07:00,-1 days +19:09:13,4.846389
2016-01-01 12:45:00-07:00,2016-01-01 12:45:00-07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,...,3.8,6.7,11.30,5.3,4.7,5.1,3.9,2016-01-01 17:30:47-07:00,-1 days +19:14:13,4.763056
2016-01-01 12:50:00-07:00,2016-01-01 12:50:00-07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.6,...,3.9,6.7,11.40,5.3,4.7,5.1,3.9,2016-01-01 17:30:47-07:00,-1 days +19:19:13,4.679722
2016-01-01 12:55:00-07:00,2016-01-01 12:55:00-07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.6,...,3.9,6.8,11.30,5.4,4.8,5.1,4.0,2016-01-01 17:30:47-07:00,-1 days +19:24:13,4.596389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-20 17:00:00-07:00,2019-11-20 17:00:00-07:00,9.5527,10.6385,9.5911,9.8355,9.6118,9.4938,9.2789,8.3940,11.6,...,9.6,10.6,5.75,8.5,10.8,9.8,9.6,2019-11-20 17:22:05-07:00,-1 days +23:37:55,0.368056
2019-11-20 17:05:00-07:00,2019-11-20 17:05:00-07:00,9.5527,10.6385,9.5911,9.8355,9.6118,9.4938,9.2789,8.3120,11.6,...,9.6,10.6,5.80,8.5,10.8,9.8,9.6,2019-11-20 17:22:05-07:00,-1 days +23:42:55,0.284722
2019-11-20 17:10:00-07:00,2019-11-20 17:10:00-07:00,9.5527,10.6385,9.5911,9.8355,9.6118,9.4938,9.2789,8.3035,11.6,...,9.6,10.6,6.00,8.5,10.8,9.8,9.6,2019-11-20 17:22:05-07:00,-1 days +23:47:55,0.201389


In [49]:
plt.close('all')
plt.scatter(st_trim['hrs_to_sunset'], st_trim['temptrax_outside'])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
st_grouped = st_trim.groupby(pd.Grouper(freq="D"))

In [51]:
for g in st_grouped.groups:
    try:
        group = st_grouped.get_group(g)
        #print(len(group['cell_e_series_outside_ambient_C']), group['cell_e_series_outside_ambient_C'][-1], group['cell_e_series_outside_ambient_C'].min())
    except:
        print(f"no data for {g}")

no data for 2017-01-23 00:00:00-07:00
no data for 2017-01-24 00:00:00-07:00
no data for 2017-06-07 00:00:00-07:00
no data for 2019-02-06 00:00:00-07:00
no data for 2019-02-08 00:00:00-07:00
no data for 2019-02-09 00:00:00-07:00
no data for 2019-02-10 00:00:00-07:00
no data for 2019-02-11 00:00:00-07:00
no data for 2019-02-12 00:00:00-07:00
no data for 2019-02-13 00:00:00-07:00
no data for 2019-02-14 00:00:00-07:00
no data for 2019-02-15 00:00:00-07:00
no data for 2019-02-16 00:00:00-07:00
no data for 2019-02-17 00:00:00-07:00
no data for 2019-02-18 00:00:00-07:00
no data for 2019-02-19 00:00:00-07:00
no data for 2019-02-20 00:00:00-07:00
no data for 2019-02-21 00:00:00-07:00
no data for 2019-02-22 00:00:00-07:00
no data for 2019-02-23 00:00:00-07:00
no data for 2019-02-24 00:00:00-07:00
no data for 2019-02-25 00:00:00-07:00
no data for 2019-02-26 00:00:00-07:00
no data for 2019-02-27 00:00:00-07:00
no data for 2019-06-27 00:00:00-07:00
no data for 2019-07-18 00:00:00-07:00
no data for 

In [52]:
st_grouped.get_group(g)

,mst,cell_e_series_backplate_C,cell_e_series_chamber_ambient_C,cell_e_series_frontplate_C,cell_e_series_in_front_of_primary_C,cell_e_series_lower_plenum_C,cell_e_series_midplate_C,cell_e_series_outside_ambient_C,yankee_temperature,temptrax1_probe2,...,temptrax1_probe4,temptrax1_probe6,temptrax_outside,temptrax_chamber,temptrax3_probe10,temptrax3_probe11,temptrax3_probe12,sunset,sunset_diff,hrs_to_sunset
mst_i,,,,,,,,,,,,,,,,,,,,,
2019-11-20 12:25:00-07:00,2019-11-20 12:25:00-07:00,9.5527,10.6385,9.5911,9.8355,9.6118,9.4938,9.2789,8.8125,11.8,...,9.7,10.7,6.90,9.10,11.20,9.80,9.7,2019-11-20 17:22:05-07:00,-1 days +19:02:55,4.951389
2019-11-20 12:30:00-07:00,2019-11-20 12:30:00-07:00,9.5527,10.6385,9.5911,9.8355,9.6118,9.4938,9.2789,8.8600,11.8,...,9.7,10.7,6.80,9.10,11.20,9.80,9.7,2019-11-20 17:22:05-07:00,-1 days +19:07:55,4.868056
2019-11-20 12:35:00-07:00,2019-11-20 12:35:00-07:00,9.5527,10.6385,9.5911,9.8355,9.6118,9.4938,9.2789,8.9230,11.8,...,9.7,10.7,6.90,9.10,11.15,9.80,9.7,2019-11-20 17:22:05-07:00,-1 days +19:12:55,4.784722
2019-11-20 12:40:00-07:00,2019-11-20 12:40:00-07:00,9.5527,10.6385,9.5911,9.8355,9.6118,9.4938,9.2789,8.7410,11.8,...,9.7,10.7,6.90,9.00,11.20,9.90,9.7,2019-11-20 17:22:05-07:00,-1 days +19:17:55,4.701389
2019-11-20 12:45:00-07:00,2019-11-20 12:45:00-07:00,9.5527,10.6385,9.5911,9.8355,9.6118,9.4938,9.2789,8.6650,11.8,...,9.7,10.7,6.80,8.90,11.20,9.80,9.7,2019-11-20 17:22:05-07:00,-1 days +19:22:55,4.618056
2019-11-20 12:50:00-07:00,2019-11-20 12:50:00-07:00,9.5527,10.6385,9.5911,9.8355,9.6118,9.4938,9.2789,8.8445,11.8,...,9.7,10.7,6.90,8.90,11.20,9.80,9.7,2019-11-20 17:22:05-07:00,-1 days +19:27:55,4.534722
2019-11-20 12:55:00-07:00,2019-11-20 12:55:00-07:00,9.5527,10.6385,9.5911,9.8355,9.6118,9.4938,9.2789,8.9325,11.8,...,9.7,10.8,6.90,8.90,11.20,9.90,9.7,2019-11-20 17:22:05-07:00,-1 days +19:32:55,4.451389
2019-11-20 13:00:00-07:00,2019-11-20 13:00:00-07:00,9.5527,10.6385,9.5911,9.8355,9.6118,9.4938,9.2789,8.7960,11.8,...,9.7,10.8,6.80,8.90,11.10,9.85,9.7,2019-11-20 17:22:05-07:00,-1 days +19:37:55,4.368056
2019-11-20 13:05:00-07:00,2019-11-20 13:05:00-07:00,9.5527,10.6385,9.5911,9.8355,9.6118,9.4938,9.2789,8.8345,11.8,...,9.7,10.8,6.80,8.90,11.20,9.80,9.7,2019-11-20 17:22:05-07:00,-1 days +19:42:55,4.284722
